In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI



In [2]:
# 1. Load Ramayana PDF
loader = PyMuPDFLoader("data/RAMAYANA.pdf")
raw_docs = loader.load()

# 2. Define Chapter Mapping by Page Range
chapter_map = {
    "1.1 THE BIRTH OF RAMA": range(2, 3),
    "1.2 The Valiant Princes": range(3, 5),
    "1.3 SITA'S SWAYAMVAR": range(5, 7),
    "1.4 KAIKEYI AND HER WISHES": range(7, 20),
    "1.5 The demons in the forests": range(20, 23),
    "1.6 The Kidnapping of Sita": range(23, 26),
    "1.7 Rama searches for Sita": range(26, 30)
}



In [3]:
# 3. Add Chapter Info to Metadata
for doc in raw_docs:
    page = doc.metadata["page"]
    for chapter, pages in chapter_map.items():
        if page in pages:
            doc.metadata["chapter"] = chapter
            break

# 4. Split Documents
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(raw_docs)



In [ ]:
# Embedder and vectorstore
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",
                                        model_kwargs={"device": "cpu"})

vectorstore = FAISS.from_documents(docs, embedding=embedding_model)

# 6. Setup Retriever with Metadata Tracing
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})



/tmp/ipykernel_30293/3305182451.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [5]:
# 7. Define Prompt for Story-style Answering
prompt_template = PromptTemplate.from_template("""
You are a storyteller known for answering questions based on Indian epics.
Use the context below to answer as a story. 
Also, cite the page number and chapter in brackets like this: [Page 2, Chapter: THE BIRTH OF RAMA].

### Context:
{context}

### Question:
{question}

### Answer:
""")



In [6]:
# 8. Setup QA Chain with Audit Trail Enabled
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4"),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)

# 9. Ask a Question with Full Traceability
query = "How was Lord Rama born and what was his family background?"
result = qa_chain(query)



/tmp/ipykernel_30293/3430665803.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(model_name="gpt-4"),
/tmp/ipykernel_30293/3430665803.py:12: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(query)


In [7]:
# 10. Print Final Answer
print("\n🧾 Answer:\n")
print(result["result"])

# 11. Print Audit Trail
print("\n📚 Audit Trail:\n")
for doc in result["source_documents"]:
    print(f"📄 Page: {doc.metadata.get('page')}")
    print(f"📘 Chapter: {doc.metadata.get('chapter', 'N/A')}")
    print(f"📂 Source: {doc.metadata.get('source')}")
    print(f"📝 Snippet: {doc.page_content[:200]}...\n")


🧾 Answer:

Lord Rama was born in the magnificent city of Ayodhya, which was situated on the banks of the river Sarayu in Kosala Country. The city was known for its wide roads, massive structures, stunning parks, and radiant shops. The people living there enjoyed a joyful and satisfied life, thanks to their benevolent ruler, King Dasharatha.

King Dasharatha, a man deeply caring for his subjects, had three wives - Kaushalya, Sumitra, and Kaikeyi. It was in this royal household that Lord Rama was born, destined to become the king of Ayodhya.

Rama was not an only child. He had three brothers Lakshmana, Bharatha and Shatrugna. Together, they grew up in Ayodhya, learning the ways of royalty and the world.

In due time, circumstances led to Rama being crowned as the king of Ayodhya, with his wife Sita by his side on the throne. His brothers, Lakshmana, Bharatha, and Shatrugna, stood behind them, supporting their reign. The people of Ayodhya rejoiced at this event, and their happiness knew 